In [ ]:
####################################################################
#                Clustering of diffusion processes                 #
####################################################################

In [4]:
#####################################################################
# Libraries
#####################################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time_series_library import *
from stochastic_library import *
from bspline_library import *
import scipy.interpolate as si
import scipy.cluster as sc

In [ ]:
#------------------------------------------------------------------#
# Compute the B-spline basis functions for the support [x_init,x_end]
#------------------------------------------------------------------#
k_final = 4 #13        # Order of the B-spline (number of constants of the polynomial)
p = 10 #50             # Number of basis = m+L-1
L = p-k_final+1        # Number of internal knots

# Support
x_init = 0. 
x_end  = 1

# Knots
knots     = np.linspace( x_init , x_end , num = L+1, endpoint = True )  # Internal knots
knots_ext = np.r_[[x_init]*(k_final-1) , knots , [x_end]*(k_final-1)]   # Add boundary knots

# Matrix representation 
Mk               = BSpline_BasisMatrix_M( k_final , x_init , x_end , L )
Jk , Nk_tilde    = BSpline_BasisMatrix_N( k_final , L , Mk )
ivl , bspl_curve = BSpline_curves( k_final , knots , Nk_tilde )
BSpline_plot( ivl , bspl_curve )

#------------------------------------------------------------------#
# Orthonormalization (via Cholesky factorization)
#------------------------------------------------------------------#
Gram_mat = BSpline_GramMatrix( k_final , knots , Nk_tilde )
Nk       = BSpline_CholOrthogon( Gram_mat , Nk_tilde , L )
ivl , orthbspl_curve = BSpline_curves( k_final , knots , Nk )
BSpline_plot( ivl , orthbspl_curve )

In [ ]:
#------------------------------------------------------------------#
# Define drift and diffusion functions
#------------------------------------------------------------------#
# drifts_fcn = [ (lambda x: 1-2*x) , (lambda x: 1-2*x) , (lambda x: 2*x-1) ,
#               (lambda x: 1.5*(0.7-x)) , (lambda x: 1.5*(0.7-x)) ,
#               (lambda x: 1.5*(0.3-x)) , (lambda x: 1.5*(0.3-x)) ,
#               (lambda x: 1-2*x) ,
#               (lambda x: 1.5*(0.7-x)) ,
#               (lambda x: 5*(0.05-x)) ]

# diffusions_fcn = [ (lambda x: 0.5+2*x*(1-x)) , (lambda x: 0.5+2*x*(1-x)) , (lambda x: 0.5+2*x*(1-x)) ,
#                    (lambda x: np.sqrt(0.55*x*(1-x))) , (lambda x: np.sqrt(0.55*x*(1-x))) ,
#                    (lambda x: np.sqrt(0.55*x*(1-x))) , (lambda x: np.sqrt(0.55*x*(1-x))) ,
#                    (lambda x: np.sqrt(0.1*x*(1-x))) ,
#                    (lambda x: np.sqrt(0.1*x*(1-x))) ,
#                    (lambda x: np.sqrt(0.8*x*(1-x))) ]

drifts_fcn = [ (lambda x: 1-2*x) , (lambda x: 1-2*x) , 
              (lambda x: 1.5*(0.3-x)) , (lambda x: 1.5*(0.3-x)) ,
              (lambda x: 5*(0.05-x)) , (lambda x: 5*(0.05-x)) ]

diffusions_fcn = [ (lambda x: 0.5+2*x*(1-x)) , (lambda x: 0.5+2*x*(1-x)) , 
                  (lambda x: np.sqrt(0.8*x*(1-x))) , (lambda x: np.sqrt(0.8*x*(1-x))) , 
                  (lambda x: np.sqrt(0.1*x*(1-x))) , (lambda x: np.sqrt(0.1*x*(1-x))) ]

#------------------------------------------------------------------#
# Stochastic process simulation
#------------------------------------------------------------------#
t0 = 0
tf = 10
N  = 1000*(tf-t0)

sim = [None] * len(drifts_fcn)
for i in range(len(drifts_fcn)):
    # Simulate using Milstein method
    ts , Xt = Milstein_method( t0 , tf , tf*N , np.random.random() , drifts_fcn[i] , diffusions_fcn[i] )

    # Save in Dataframe with 'Time' and 'Values'
    sim[i]           = pd.DataFrame( index = list(range(len(ts))) , columns = list( [ 'Time' , 'Values' ] ) )
    sim[i]['Time']   = list(ts)
    sim[i]['Values'] = list(Xt)

#------------------------------------------------------------------#
# Plot
#------------------------------------------------------------------#
fig, axs = plt.subplots(len(sim))

for i in range(len(axs)):
    axs[i].plot(sim[i]['Time'], sim[i]['Values'])
    axs[i].grid(True)

plt.show()

In [5]:
#------------------------------------------------------------------#
# Build the observations vectors with lower sample rate
#------------------------------------------------------------------#
step_obs = 0.01
obs      = [None] * len(sim)
for i in range(len(sim)):
    tq , Xq = Observation_sampler( t0 , tf , step_obs , sim[i]['Time'] , sim[i]['Values'] )
    
    # Save in Dataframe with 'Time' and 'Values'
    obs[i]           = pd.DataFrame( index = list(range(len(tq))) , columns = list( [ 'Time' , 'Values' ] ) )
    obs[i]['Time']   = list(tq)
    obs[i]['Values'] = list(Xq)

#------------------------------------------------------------------#
# Plot
#------------------------------------------------------------------#
# fig, axs = plt.subplots(len(obs))

# for i in range(len(axs)):
#     axs[i].plot(obs[i]['Time'], obs[i]['Values'])
#     axs[i].grid(True)

# plt.show()

In [ ]:
from tqdm import tqdm

#------------------------------------------------------------------#
# Compute the estimated transition operator matrix P
#------------------------------------------------------------------#
N_obs = len(obs)

P = [None] * N_obs
for idxObs in tqdm( range( N_obs ) ):
    P_tmp = Probability_TransitionOperator( obs[idxObs]['Values'] , k_final , x_init , x_end , L , Nk )
    P[idxObs] = P_tmp.copy( )

In [ ]:
#------------------------------------------------------------------#
# Compute the distance matrix from transition operator matrix P
#------------------------------------------------------------------#

D = np.zeros((len(obs),len(obs)))

for i in tqdm( range(0,len(obs)-1) ):
    k = i+1
    while ( k <= len(obs)-1 ):
        for P_row in range(p):
            for P_col in range(p):
                D[i][k] = D[i][k] + np.absolute( P[i][P_row,P_col] - P[k][P_row,P_col] )#/1e26
                D[k][i] = D[i][k]
        k = k+1

#------------------------------------------------------------------#
# Compute clusters based on D and plot dendrogram
#------------------------------------------------------------------#
import scipy.spatial.distance as ssd
# convert the redundant n*n square matrix form into a condensed nC2 array
distArray = ssd.squareform(D) # distArray[{n choose 2}-{n-i choose 2} + (j-i-1)] is the distance between points i and j

Z = sc.hierarchy.linkage( distArray , method='single' )

fig = plt.figure(figsize=(10, 10))
dn = sc.hierarchy.dendrogram(Z,labels=[ 'X1' , 'X2' , 'X3' , 'X4' , 'X5' , 'X6' ])
plt.show()